In [1]:
from pathlib import Path
import sys

dir_src = Path.cwd().parent.parent/'src'
sys.path.insert(0, str(dir_src))

In [2]:
import os

from arcgis.gis import GIS
from dotenv import load_dotenv, find_dotenv
from modeling import Country

# load environment variables from .env
load_dotenv(find_dotenv())

In [3]:
gis_ent= GIS(os.getenv('ESRI_PORTAL_URL'), username=os.getenv('ESRI_PORTAL_USERNAME'), password=os.getenv('ESRI_PORTAL_PASSWORD'))

gis_ent

GIS @ https://geoai-ent.bd.esri.com/portal/ version:8.2

In [4]:
ent_usa = Country('USA', gis_ent)

ent_usa

<modeling.Country - USA (GIS at https://geoai-ent.bd.esri.com/portal/ logged in as jmccune)>

In [5]:
%%time
ent_df = ent_usa.cbsas.get('seattle').mdl.block_groups.get()

ent_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2480 entries, 0 to 2479
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   ID      2480 non-null   object  
 1   NAME    2480 non-null   object  
 2   SHAPE   2480 non-null   geometry
dtypes: geometry(1), object(2)
memory usage: 58.2+ KB
Wall time: 4.63 s


In [6]:
ev = ent_usa.enrich_variables

enrich_variables = ev[
    (ev.data_collection.str.lower().str.contains('key'))  # get the key variables
    & (ev.name.str.endswith('CY'))                     # just current year (2019) variables
].reset_index(drop=True)

enrich_variables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               20 non-null     object
 1   alias              20 non-null     object
 2   data_collection    20 non-null     object
 3   enrich_name        20 non-null     object
 4   enrich_field_name  20 non-null     object
 5   description        20 non-null     object
 6   vintage            20 non-null     object
 7   units              20 non-null     object
dtypes: object(8)
memory usage: 1.4+ KB


In [7]:
enrich_variables.sample(5)

,name,alias,data_collection,enrich_name,enrich_field_name,description,vintage,units
5,MEDHINC_CY,2020 Median Household Income,KeyUSFacts,KeyUSFacts.MEDHINC_CY,KeyUSFacts_MEDHINC_CY,2020 Median Household Income (Esri),2020,currency
4,AVGHHSZ_CY,2020 Average Household Size,KeyUSFacts,KeyUSFacts.AVGHHSZ_CY,KeyUSFacts_AVGHHSZ_CY,2020 Average Household Size (Esri),2020,count
13,AVGVAL_CY,2020 Average Home Value,KeyUSFacts,KeyUSFacts.AVGVAL_CY,KeyUSFacts_AVGVAL_CY,2020 Average Home Value (Esri),2020,currency
11,VACANT_CY,2020 Vacant Housing Units,KeyUSFacts,KeyUSFacts.VACANT_CY,KeyUSFacts_VACANT_CY,2020 Vacant Housing Units (Esri),2020,count
15,HHGRW10CY,2010-2020 Growth Rate: Households,KeyUSFacts,KeyUSFacts.HHGRW10CY,KeyUSFacts_HHGRW10CY,2010-2020 Households: Annual Growth Rate (Esri),2020,pct


In [8]:
%%time
out_df = ent_df.mdl.enrich(enrich_variables)

out_df

Wall time: 11.6 s


,ID,NAME,TOTPOP_CY,GQPOP_CY,DIVINDX_CY,TOTHH_CY,AVGHHSZ_CY,MEDHINC_CY,AVGHINC_CY,PCI_CY,...,VACANT_CY,MEDVAL_CY,AVGVAL_CY,POPGRW10CY,HHGRW10CY,FAMGRW10CY,DPOP_CY,DPOPWRK_CY,DPOPRES_CY,SHAPE
0,530330012001,530330012.001,1466,0,55.7,588,2.49,120824,155449,62349,...,27,477551,614634,1.47,1.30,1.72,1524,782,742,"{""rings"": [[[-122.31243049957185, 47.704870499..."
1,530330012002,530330012.002,1233,0,73.5,553,2.23,63251,98980,44392,...,30,833333,800833,0.69,0.53,1.11,1101,573,528,"{""rings"": [[[-122.3178410004258, 47.7076489995..."
2,530330032001,530330032.001,1171,0,26.6,559,2.09,117030,147031,70188,...,3,878788,938628,2.39,2.23,2.86,848,427,421,"{""rings"": [[[-122.38754700019909, 47.675934999..."
3,530330032002,530330032.002,1405,0,46.4,775,1.81,92628,106468,58728,...,34,699728,838340,2.01,1.87,2.66,1203,769,434,"{""rings"": [[[-122.38758899913357, 47.671643999..."
4,530330032003,530330032.003,1868,0,40.5,1118,1.67,84004,101288,60621,...,3,498947,582927,5.28,5.11,5.98,1988,1182,806,"{""rings"": [[[-122.38758899913357, 47.669455999..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,530610529043,530610529.043,1367,0,52.5,463,2.95,98532,102821,34825,...,12,305163,285127,0.64,0.61,0.58,936,291,645,"{""rings"": [[[-122.14943799870154, 48.064500999..."
2476,530610529044,530610529.044,1228,0,47.4,412,2.98,87379,100806,33821,...,11,329961,316621,0.01,-0.02,-0.06,934,335,599,"{""rings"": [[[-122.14359799983554, 48.059699000..."
2477,530619400011,530619400.011,1543,0,72.4,535,2.88,84472,106721,37003,...,16,344932,421106,0.57,0.55,0.46,1161,227,934,"{""rings"": [[[-122.22630799998365, 48.097295999..."
2478,530619400012,530619400.012,1785,0,66.6,571,3.13,76932,90127,28831,...,22,304135,438521,0.61,0.58,0.49,1804,793,1011,"{""rings"": [[[-122.19474699952485, 48.075618000..."
